# Stable Diffusion (Colab + VS Code friendly)

This notebook is a cleaned, minimal flow for running Stable Diffusion.
- Install once
- Verify GPU
- Load model once
- Generate images

If you are connecting from VS Code to a Colab runtime, execution still happens on Colab's GPU.

In [ ]:
print("Installing dependencies...")
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install diffusers transformers accelerate
print("Done.")

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

print(f"Torch: {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA OK: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA not available. In Colab, set Runtime > Change runtime type > GPU.")

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading {model_id} on {device}...")
pipeline = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)
pipeline = pipeline.to(device)
print("Model loaded.")

In [ ]:
prompt = "A high-quality photo of an astronaut riding a horse on the moon, cinematic, 4k"
print(f"Prompt: {prompt}")

with torch.no_grad():
    image = pipeline(prompt, num_inference_steps=25).images[0]

image

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

text_input = widgets.Text(
    value="A futuristic city at sunset with flying cars and neon lights",
    description="Prompt:"
)

generate_button = widgets.Button(description="Generate Image")
output_area = widgets.Output()

def on_button_clicked(_):
    with output_area:
        clear_output()
        print(f"Generating: {text_input.value}")
        with torch.no_grad():
            img = pipeline(text_input.value, num_inference_steps=25).images[0]
        display(img)

generate_button.on_click(on_button_clicked)
display(text_input, generate_button, output_area)